In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import importlib
import xarray as xr
import scipy.spatial as spatial

In [ ]:
import beam_utils
importlib.reload(beam_utils)
import tri_mesh
importlib.reload(tri_mesh)

# Beam Integration using using xarray datasets

This demonstrate beam integration using an xarray dataset for the beam data.


## Create Interpolating Function from  Simulation Data

In [ ]:
viz_path = r"C:\Users\Huckaby\Desktop\MHD\Simulations\viz_example" 
#print(os.listdir(viz_path))
fname = "frontCyl_plasma.vtk"
fname = "frontCyl.vtk"
fname = os.path.join(viz_path, fname)
f_axi = beam_utils.new_soln_interpolator(fname, do_plot=False)

## Construct Beam Dataset

### Beam Mesh

In [ ]:
x_exit = 207.910 * 1e-3
x_min = 0.25
x_center = x_min
offset = np.array([-0.05,0.0,-0.1])

pos_center = np.array([x_center,0,0])
pos_source = pos_center + offset
pos_target = pos_center - offset

beam_axis = -offset
jet_axis = (1,0,0)
shat = beam_utils.coordinate_tensor(beam_axis, jet_axis)
L_beam = beam_utils.mag(offset)*2.0

x_beam = np.linspace(0,L_beam)
d_beam = 1e-2
# creates pos[:,:], mesh:pv.mesh, ds:xr.dataset, 
tri_out = tri_mesh.new_cylinder(x=x_beam,r=d_beam/2.0,n_theta=60)


In [ ]:
fig, ax = plt.subplots(1,3,sharex=True,sharey=True)
for a in ax:
    a.set_aspect(1.0)
circle = tri_mesh.new_disk(n_theta=50, angle=30.0, do_plot=False)
v = circle["vertices"]
vor = spatial.Voronoi(v)
hull = spatial.ConvexHull(v,qhull_options='QG-100')
for i in [0,1]:
    spatial.voronoi_plot_2d(vor, ax=ax[i], line_width=0.5, point_size=1.0, show_vertices=False, show_points=i)

for visible_facet in hull.simplices[hull.good]:
    ax[1].plot(hull.points[visible_facet, 0],
            hull.points[visible_facet, 1],
            color='violet',
            lw=1)

for i in [0,2]:
    ax[i].triplot(v[:,0], v[:,1], circle["triangles"], linewidth=0.5)
fig.set_size_inches(10,10)

### **Figure** Mesh and Voronoi Cross-section

Left - Voronoi and Mesh
Center - Voronoi w/ Mesh vertices and Convex Hull (purple)
Right - Mesh edges

Since the beam is being integration along the vertices, there are two options to reconstruct the 

1. "Trapazoidal Rule"
    I_face = \sum_i A_i I_i
    I_i = \sum_k I_{j = vertex[i,k]}/n_vertex
    i - triangles
    j - verticies
    v[k] - vertrex k of triangle i

1a. Invert the summation
    I_face = \sum_j Ahat_j I_j
    Ahat_j = \sum_k \sum_i (A_i/3) delta_{j,vertex[i,k]}
      
2. "Voronoi Cell"
     I_face = \sum_i Ahat_j I_j
     Ahat_j <- area of the Voronio cell for the interior
     for exterior cells, the area inside the boundary.

1a is probably the best approach, since Ahat_j can be calcualted once a resused for all wavelengths   

In [ ]:
def transform_beam(ds, shat):
    ds["pos_beam"] = ds["pos"]*1.0
    ds["pos"][:] = np.matmul(ds["pos_beam"].to_numpy(),shat) + pos_source

### Create Dataset

Add concentration and temperature to the dataset

In [ ]:
ds = tri_out["ds"]
transform_beam(ds, shat)

In [ ]:
f_out = f_axi(ds["pos"])
ds["T"] = ("s","ray"), f_out[:,:,0].T
ds["K"] = ("s","ray"), f_out[:,:,1].T

In [ ]:
ds

In [ ]:
ds.coords["x"] = ("s", "ray"), ds["pos"].data[:,:,0]
ds.coords["y"] = ("s", "ray"), ds["pos"].data[:,:,1]
ds.coords["z"] = ("s", "ray"), ds["pos"].data[:,:,2]
ds

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect(1.0)
ds["T"].plot(x="x",y="z", ax=ax)

In [ ]:
### **Figure** Temperature Contour On Beam

## Adaptive Refinement of Beam Mesh

In [ ]:
ds_diff = ds.diff("s")

In [ ]:
v_min = ds.min(dim="s")
v_max = ds.max(dim="s")
w_small = 1e-6
wv_small = 1e-12
weights = (v_max - v_min) + (v_max + v_min)*w_small + wv_small
adapt_vars = "T","K"
rtol = 1e-2

In [ ]:
max_iter = 10
ds_new = ds.copy()
for i in range(max_iter):
    ds_diff = ds.diff("s")
    mask = (ds_diff > rtol*weights).any("ray")
    i_new = np.concatenate([ np.where(mask[v].to_numpy())[0] for v in adapt_vars ])
    i_new = np.unique(i_new)
    
    s_new = (ds.s[i_new] + ds.s[i_new+1])*0.5
    #coords = ds.coords.copy()
    coords = {"s": s_new}
    #ds_new = ds.DataSet(coords)

    break

In [ ]:
ds_new = beam_utils.adapt_beam(ds, f_axi)

In [ ]:
fig, ax = plt.subplots(1,2,sharex=True, sharey=True)
out = ax[0].plot(ds["x"].sel(ray=0),ds["T"].sel(ray=0),'.-')
ax[0].set_title("coarse N={}".format(len(ds["x"])))
out = ax[1].plot(ds_new["x"].sel(ray=0),ds_new["T"].sel(ray=0),'.',alpha=0.05, label=0)
out = ax[1].plot(ds_new["x"].sel(ray=0),ds_new["T"].sel(ray=5),'.',alpha=0.05, label=5)
ax[1].legend(title="ray#")
ax[1].set_title("adapt N={}".format(len(ds_new["x"])))
ax[1].set_xlabel("x [m]")
ax[0].set_xlabel("x [m]")
ax[0].set_ylabel("T [K]")

### **Figure** Temperature Profile of the first ray in the beam

x - is the coordinate parallel to jet axis

In [ ]:
fig, ax = plt.subplots(2,2,sharex=True,sharey=True)
for a in ax.flat:
    a.set_aspect(1.0)
ds_new["T"].plot(x="x",y="z",ax=ax[0,0])
ds["T"].plot(x="x",y="z",ax=ax[0,1])
ds_new["K"].plot(x="x",y="z",ax=ax[1,0])
ds["K"].plot(x="x",y="z",ax=ax[1,1])
ax[0,0].set_title("adapt")
ax[0,1].set_title("fixed")
fig.tight_layout()

### **Figure** Comparison on Bean Temperature and Potassium contours

## Beam Integration

### Experimental Source 

In [ ]:
data_path = r"C:\Users\Huckaby\Desktop\MHD\Simulations\viz_example"
fname = "ds_calib.cdf"
fname = os.path.join(data_path,fname)
source = beam_utils.read_intensity_wavelength_function(fname, do_plot=False)
u = source["diff"].isel(time=0).drop("time")
for mp in u.mp.data:
    v = u.sel(mp=mp)
    plt.plot(v["wavelength"], v, label=mp)
plt.xlabel("wavelength [nm]")
plt.ylabel("intensity [W]")
plt.legend()

### **Figure** - Source Intensity

### Arbitrary Gaussian Absoprtion Cross Section

In [ ]:
import ray_integration
Q_absorption = ray_integration.GaussAbsorption(b=[-0.3,-0.3])

In [ ]:
Q_list = beam_utils.calc_absorption(source, ds_new, Q_absorption)

In [ ]:
wave_nm = ds_new.wavelength
fig, ax = plt.subplots(2,1,sharex=True)
for T in [300,500,1500]:
    Q = np.array([Q_absorption(T, w) for w in wave_nm.to_numpy()])    
    ax[0].semilogy(wave_nm, Q)
    ax[1].plot(wave_nm, Q, label=T)
ax[1].set_xlabel("wavelength [nm]")
ax[0].set_ylabel("cross section [m^2]")
ax[1].set_ylabel("cross section [m^2]")
ax[1].legend(title="T")            

In [ ]:
ds_new["I_target"].isel(mp=0,wavelength=200).plot()

### **Figure** - Target Intensity for Each Ray

In [ ]:
import matplotlib.tri as plt_tri
def plot(ds):
    f = ds["I_target"].isel(wavelength=200, mp=0)
    f_rel = (f - f.min())/(f.max() - f.min())
    #print(z.max(), z.min())
    cmap = plt.get_cmap("viridis")
    color=cmap(f_rel)
    fig, ax = plt.subplots(1,2,sharex=True,sharey=True)
    x, z, y = ds["x"].isel(s=-1)**2, ds["z"].isel(s=-1)**2 + ds["x"].isel(s=-1), ds["y"].isel(s=-1)
    x1 = ( (x - x.min())**2 + (z - z.min())**2 )**0.5
    ax[0].scatter(x1, y, c=color)
    tri_out = plt_tri.Triangulation(x1, y)
    out = ax[1].tricontourf(tri_out, f_rel)
    cbar = plt.colorbar(out)
    #cbar.title("I")
    ax[1].set_aspect(1.0)
    ax[0].set_aspect(1.0)
    fig.tight_layout()

plot(ds_new)

### **Figure** - Target Relative Intensity Contours

In [ ]:
ds_new